# Experimento 3
En este experimento decidí usar redes neuronales, con una biblioteca llamada gensim

In [7]:
import time
import gensim
import os
import collections
import smart_open
import random
import csv
import unicodedata

def read_corpus(fname, tokens_only=False):
	global etq
	# ignorar=open("stopwords-es-master/stopwords-es.txt","r").read().splitlines()
	ignorar=[]
	with open(fname,"r") as f:
		lineas=map(lambda x:[i for i in x.split("|",1)],f.read().splitlines())
		ind=0
		for row in lineas:
			if len(row)>1 and len(row[1])!=0:
				aux=unicodedata.normalize('NFKD',row[1]).lower()
				for i in ignorar:
					aux=aux.replace(" "+i+" "," ")
				if len(aux.split())>30:
					if tokens_only:
						yield gensim.utils.simple_preprocess(aux)
					else:
						yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(aux), [ind])
					etq[ind]=row[0]
					# input(ind)
					ind+=1

def splitInv(cad,caracteres,stopwords):
	arreglo=[]
	aux=""
	cad=cad.lower()+"."
	for c in cad:
		if c not in caracteres :
			if aux!="" and aux not in stopwords:
				arreglo.append(aux)
			aux=""
		else:
			aux+=c
	return arreglo

Con la siguiente parte de código se configura y entrena el modelo, en los siguientes ejemplos uso un corpus pequeño y uno mas grande, el grande es datosLimpios.csv el cual tiene 1.6 M de contenido y el pequeño es datos.csv el cual tiene solo 879 K

In [4]:
etq={}
# stopwords=open("stopwords-es-master/stopwords-es.txt","r").read().splitlines()
stopwords=[]
train_corpus = list(read_corpus("datosLimpios.csv"))
model = gensim.models.doc2vec.Doc2Vec(
	vector_size=100,
	min_count=2,
	window=5,
	# dm=1,
	dm_concat=1,
	# dbow_words=1,
	epochs=450,
	workers=16,
	# ns_exponent=0
	)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)
texto=splitInv(open("misSintomas.txt").read(),"qwertyuiopasdfghjklzxcvbnm",stopwords)
print(texto)
vector=model.infer_vector(texto)
sim=model.docvecs.most_similar([vector],topn=4)
for i in sim:
	print("Documento Similar %s %s: " % (etq[i[0]],i[1]))
# -
texto=splitInv(open("misSintomas2.txt").read(),"qwertyuiopasdfghjklzxcvbnm",stopwords)
print(texto)
vector=model.infer_vector(texto)
sim=model.docvecs.most_similar([vector],topn=4)
for i in sim:
	print("Documento Similar %s %s: " % (etq[i[0]],i[1]))

# -
texto=splitInv(open("misSintomas3.txt").read(),"qwertyuiopasdfghjklzxcvbnm",stopwords)
print(texto)
vector=model.infer_vector(texto)
sim=model.docvecs.most_similar([vector],topn=4)
for i in sim:
	print("Documento Similar %s %s: " % (etq[i[0]],i[1]))

['tengo', 'dolor', 'de', 'espalda', 'baja', 'por', 'las', 'noches', 'mejora', 'con', 'el', 'ejercicio', 'y', 'antibioticos', 'tuve', 'una', 'uveitis', 'en', 'mi', 'ojo', 'izquierdo', 'ademas', 'de', 'dolor', 'en', 'el', 'cuello', 'y', 'articulaciones', 'de', 'las', 'manos', 'ademas', 'tengo', 'hla', 'b', 'positivo']
Documento Similar anorexia_nerviosa.html  0.3453219532966614: 
Documento Similar enfermedad-de-whipple.html  0.344423770904541: 
Documento Similar espondilitis_anquilosante.html  0.33348092436790466: 
Documento Similar sindrome-de-marfan.html  0.3235524296760559: 
['nariz', 'tapada', 'estornudos', 'aveces', 'tengo', 'en', 'el', 'paladar', 'mucho', 'moco']
Documento Similar paperas.html  0.4160977602005005: 
Documento Similar polipos_intestinales.html  0.40721043944358826: 
Documento Similar ronquido.html  0.40517380833625793: 
Documento Similar tos_ferina.html  0.3965616822242737: 
['difficultad', 'para', 'respirar', 'tos', 'fatiga', 'produccion', 'excesiva', 'de', 'moco']


Como se puede ver este modelo sirve mejor cuando se le dan entre 500 y 600 epoch,

In [9]:
train_corpus = list(read_corpus("../datos.csv"))
model = gensim.models.doc2vec.Doc2Vec(
	vector_size=100,
	min_count=5,
	window=2,
	dm=0,
	dm_concat=1,
	# dbow_words=0,
	epochs=300,
	workers=16,
	ns_exponent=1.125
	)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)
texto=splitInv(open("misSintomas.txt").read(),"qwertyuiopasdfghjklzxcvbnm",stopwords)
print(texto)
vector=model.infer_vector(texto)
sim=model.docvecs.most_similar([vector],topn=4)
for i in sim:
	print("Documento Similar %s %s: " % (etq[i[0]],i[1]))
# -
texto=splitInv(open("misSintomas2.txt").read(),"qwertyuiopasdfghjklzxcvbnm",stopwords)
print(texto)
vector=model.infer_vector(texto)
sim=model.docvecs.most_similar([vector],topn=4)
for i in sim:
	print("Documento Similar %s %s: " % (etq[i[0]],i[1]))

# -
texto=splitInv(open("misSintomas3.txt").read(),"qwertyuiopasdfghjklzxcvbnm",stopwords)
print(texto)
vector=model.infer_vector(texto)
sim=model.docvecs.most_similar([vector],topn=4)
for i in sim:
	print("Documento Similar %s %s: " % (etq[i[0]],i[1]))

['tengo', 'dolor', 'de', 'espalda', 'baja', 'por', 'las', 'noches', 'mejora', 'con', 'el', 'ejercicio', 'y', 'antibioticos', 'tuve', 'una', 'uveitis', 'en', 'mi', 'ojo', 'izquierdo', 'ademas', 'de', 'dolor', 'en', 'el', 'cuello', 'y', 'articulaciones', 'de', 'las', 'manos', 'ademas', 'tengo', 'hla', 'b', 'positivo']
Documento Similar espondilitis-anquilosante.html  0.44475024938583374: 
Documento Similar glaucoma.html  0.42014431953430176: 
Documento Similar incontinencia-urinaria.html  0.4023008346557617: 
Documento Similar ojo-seco.html  0.3839343190193176: 
['nariz', 'tapada', 'estornudos', 'aveces', 'tengo', 'en', 'el', 'paladar', 'mucho', 'moco']
Documento Similar dismorfofobia.html  0.6507455706596375: 
Documento Similar gastritis.html  0.5277287364006042: 
Documento Similar bronquitis.html  0.5262576341629028: 
Documento Similar faringitis.html  0.5222646594047546: 
['difficultad', 'para', 'respirar', 'tos', 'fatiga', 'produccion', 'excesiva', 'de', 'moco']
Documento Similar enf

Con este modelo ya entrenado, podemos predecir que enfermedad es 

## Conclusiones
Como conclusión de este experimento, pudimos ver que no funciona realmente como queremos, entre más grande sea nuestro corpus, se necesitan más epoch para entrenar, además de esto es necesario calcular un rango específico de las epoch ya que si son muchas, el sistema se sobre entrena, y dado que no son datos balanceados, es decir no hay el mismo número de enfermedades relacionadas con los huesos, la garganta, la cabeza etc., por lo tanto este sistema está más desviado a predecir ciertar enfermedades.